In [1]:
import numpy as np
import sys
from bs4 import BeautifulSoup as BSoup
import requests

In [12]:
r = requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247")

In [34]:
soup = BSoup(r.text, "lxml")
selectFields = soup.find_all("select")

In [59]:
infoField=selectFields[0].find("option", text="Informatique")["value"]

In [60]:
bsSem1=selectFields[2].find("option", text="Bachelor semestre 1")["value"]

In [61]:
bsSem2=selectFields[2].find("option", text="Bachelor semestre 6")["value"]

In [62]:
infoFieldParam=selectFields[0]["name"]

In [63]:
semParam=selectFields[2]["name"]

In [43]:
link="http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=71297531&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=249847&ww_x_PERIODE_PEDAGO=249108"

In [44]:
r2= requests.get(link)

In [50]:
htmlradiobutton=soup.find("input", type="radio")

In [52]:
htmlradiobutton["name"]

'ww_i_reportModelXsl'

In [56]:
htmlradiobutton["value"]

'133685270'

In [57]:
l1="http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=71297531&ww_i_reportModel=133685247"

In [58]:
l2=l1+"&"+htmlradiobutton["name"]+"="+htmlradiobutton["value"]

In [64]:
l3=l2+"&"+infoFieldParam+"="+infoField

In [65]:
l4=l3+"&"+semParam+"="+bsSem1

In [66]:
r2= requests.get(l4)

In [67]:
r2.text

u'<html><head><META http-equiv="Content-Type" content="text/html; charset=iso-8859-1"><link rel="stylesheet" type="text/css" href="gedpublicreports.css?ww_x_path=Gestac.Moniteur.Style"></head><body bgcolor="#ffffff" marginheight="0" marginwidth="5" link="#666666" vlink="#666666" alink="#666666"><fieldset style="text-align:right; width:40%; position:relative; margin-right: 10px;float:right; border: 0; padding: 0 0 8px 0;"><a style="color:#990033;" href="!GEDREPORTS.html?ww_x_GPS=71297531&amp;ww_i_reportModel=133685247&amp;ww_i_reportModelXsl=133685270&amp;ww_x_UNITE_ACAD=249847&amp;ww_x_PERIODE_PEDAGO=249108">Identification pour acc\xe9der aux e-mails<br>Login to access email adresses</a></fieldset><script>\n  function mailList(x) {\n   var vtop = (screen.height-200)/2;\n   var vleft=(screen.width-600)/2;\n   var w=open("", "emaillist", "Scrollbars=1,resizable=1,width=600,height=200,top="+vtop+",left="+vleft);\n   w.document.write(x);\n   w.document.close();\n   w.focus();\n  }\n </scri